In [ ]:

# Fake News Detection using BERT (Optimized for Speed)

# 📦 Install necessary libraries (if not already installed)
!pip install -q transformers datasets

# 📚 Import modules
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

# 🧹 Load and preprocess dataset
fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
real = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

# Label and merge
fake['label'] = 0
real['label'] = 1
df = pd.concat([fake, real]).sample(frac=1).reset_index(drop=True)
df['text'] = df['title'] + ' ' + df['text']

# Reduce size for faster training
df = df.sample(n=2000, random_state=42).reset_index(drop=True)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# 🧼 Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

# 📦 Dataset preparation
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, list(y_train))
test_dataset = FakeNewsDataset(test_encodings, list(y_test))

# ⚙️ Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# ⚙️ Training configuration
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch"
)

# 🧠 Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# 🚀 Train the model
trainer.train()

# 🧪 Evaluation
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

# 🧾 Report
print(classification_report(y_test, pred_labels))
